In [30]:
import pandas as pd

cm_features = pd.read_csv('data/cm_features_with_name.csv')
# sort by month_id and country_id
cm_features = cm_features.sort_values(by=['month_id', 'country_id'])
cm_features

,month_id,country_id,gleditsch_ward,ged_sb,ged_ns,ged_os,acled_sb,acled_sb_count,acled_os,ged_sb_tsum_24,...,services_efficiency_t48,general_efficiency_t48,water_stress_t48,renewable_internal_pcap_t48,renewable_pcap_t48,splag_1_decay_ged_sb_5,splag_1_decay_ged_os_5,splag_1_decay_ged_ns_5,country,date
0,121,1,110,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,27.952694,1.380676,3.333333,315260.986388,354505.092577,0.093750,1.034032,0.093750,Guyana,1990-01-01
1,121,2,115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,57.289771,3.922495,2.950609,264606.831667,264606.831667,0.062500,0.062500,0.062500,Suriname,1990-01-01
2,121,3,52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,35.755839,36.015231,17.956469,3244.939711,3244.939711,0.000000,0.000000,0.000000,Trinidad and Tobago,1990-01-01
3,121,4,101,0.0,0.0,12.0,0.0,0.0,0.0,0.0,...,28.946372,26.864392,2.249173,45295.968642,74555.476336,1.062500,1.062500,0.093750,Venezuela,1990-01-01
4,121,5,990,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Samoa,1990-01-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71637,502,242,510,0.0,0.0,1.0,0.0,0.0,2.0,30.0,...,41.083011,6.915093,12.963241,1536.763246,1761.240449,4.412313,3.889290,2.105145,Tanzania,2021-10-01
71638,502,243,600,0.0,0.0,0.0,2.0,2.0,0.0,5.0,...,54.588834,8.417189,50.751212,815.035895,815.035895,0.823197,0.049630,0.060734,Morocco,2021-10-01
71639,502,244,435,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,32.697300,3.774539,13.246217,93.401859,2661.952986,1.844790,1.055986,1.065787,Mauritania,2021-10-01
71640,502,245,625,3.0,3.0,9.0,2.0,1.0,42.0,82.0,...,53.047036,2.620623,118.656388,98.007034,926.166474,5.816131,4.897757,4.263890,Sudan,2021-10-01


In [31]:
import pandas as pd

# cm_features is initial DataFrame with columns ['month_id', 'country_id', 'country_name', 'date', ...]

# Initialize a list to collect dyadic pair data
dyadic_data = []

# Features to exclude from the dyadic DataFrame
features_exclude = ['date', 'country_id', 'month_id', 'country']  # Excluding 'country_name' to handle separately

# Features to create ratios for
features_for_ratios = ['wdi_sp_pop_totl', 'ged_sb', 'wdi_sp_dyn_imrt_in', 'vdem_v2x_ex_military', 'wdi_ms_mil_xpnd_zs']

# Iterate over each month only once
for month in cm_features['month_id'].unique():
    print("Processing month: ", month)
    # Filter the DataFrame for the current month only once
    cm_features_month = cm_features[cm_features['month_id'] == month]
    date = cm_features_month['date'].values[0]

    # Get unique countries in the month
    countries = cm_features_month['country_id'].unique()

    # Generate all unique pairs of countries without repeating pairs
    for i in range(len(countries)):
        print("Processing country: ", i)
        for j in range(i + 1, len(countries)):
            country_i = countries[i]
            country_j = countries[j]

            # Filter data for each country and drop excluded features
            data_i = cm_features_month[cm_features_month['country_id'] == country_i]
            data_j = cm_features_month[cm_features_month['country_id'] == country_j]

            # Get country names for both countries
            country_name_i = data_i['country'].values[0]
            country_name_j = data_j['country'].values[0]

            # Drop excluded features
            data_i = data_i.drop(columns=features_exclude)
            data_j = data_j.drop(columns=features_exclude)

            # Initialize dyad dictionary with month, country IDs, and country names
            dyad = {
                'month_id': month,
                'date': date,
                'country_id_a': country_i,
                'country_id_b': country_j,
                'a_country_name': country_name_i,
                'b_country_name': country_name_j
            }

            # Add data for country_i and country_j to dyad, prefix with 'a_' and 'b_'
            for feature in data_i.columns:
                dyad[f'a_{feature}'] = data_i[feature].values[0]  # Assuming one row per country per month
            for feature in data_j.columns:
                dyad[f'b_{feature}'] = data_j[feature].values[0]

            # Calculate and add ratios for specified features
            for feature in features_for_ratios:
                if feature in data_i.columns and feature in data_j.columns:
                    value_i = data_i[feature].values[0]  # Assuming one row per country per month
                    value_j = data_j[feature].values[0]
                    # Adjust for division by zero
                    if value_j == 0:
                        value_j = 1
                    ratio = value_i / value_j
                    dyad[f'ratio_{feature}'] = ratio

            # Append the completed dyad to the list
            dyadic_data.append(dyad)


In [ ]:
dyadic_df = pd.DataFrame(dyadic_data)
dyadic_df.info()